# Bond Ladder Analysis

This notebook demonstrates the bond ladder simulator for fixed income portfolio construction.

## Overview

A bond ladder is a portfolio of bonds with staggered maturity dates, providing:
- Regular income from maturing bonds
- Reduced interest rate risk compared to single maturity
- Liquidity at regular intervals
- Opportunity to reinvest at current rates

## Bond Ladder Simulator

The simulator models:
- Bond purchases at different maturities (1-20 years)
- Present value calculations using historical yield curves
- Maturity proceeds and reinvestment
- Total return over time
- Comparison with bond fund alternatives (TLT, IEF, SHY)

## Use Cases

1. **Retirement Income**: Create predictable cash flows
2. **Interest Rate Risk Management**: Spread exposure across yield curve
3. **Bond vs Fund Comparison**: Analyze ladders vs bond ETFs
4. **Yield Curve Strategies**: Optimize maturity distribution

In [ ]:
# Setup
import warnings

warnings.filterwarnings("ignore")

# Set environment
import os

import pandas as pd
import plotly.graph_objects as go

os.environ["DYNACONF_ENV"] = "development"

from config import logger

logger.info("Notebook initialized")

## 1. Load Yield Curve Data

Load historical Treasury yield curve data from FRED.

In [ ]:
from finbot.utils.data_collection_utils.fred.get_fred_data import get_fred_data

# Treasury yields at different maturities
yield_symbols = {
    "DGS1": "1-Year",
    "DGS2": "2-Year",
    "DGS5": "5-Year",
    "DGS7": "7-Year",
    "DGS10": "10-Year",
    "DGS20": "20-Year",
    "DGS30": "30-Year",
}

yields = {}

for symbol, name in yield_symbols.items():
    try:
        yields[name] = get_fred_data(symbol)
        print(f"✓ Loaded {name} Treasury: {len(yields[name])} days")
    except Exception as e:
        print(f"✗ Error loading {symbol}: {e}")

# Create yield curve DataFrame
yield_df = pd.DataFrame(yields)

# Display recent yield curve
print("\nRecent Yield Curve:")
print(yield_df.tail())

## 2. Visualize Yield Curve Evolution

Plot how the yield curve has changed over time.

In [ ]:
# Plot recent yield curves
fig = go.Figure()

# Select snapshots (quarterly over 2 years)
snapshot_dates = yield_df.index[-1::63][-8:]  # Last 8 quarters

maturities = [1, 2, 5, 7, 10, 20, 30]
maturity_names = list(yield_symbols.values())

for date in snapshot_dates:
    if date in yield_df.index:
        curve = yield_df.loc[date]
        fig.add_trace(go.Scatter(x=maturities, y=curve.values, mode="lines+markers", name=date.strftime("%Y-%m-%d")))

fig.update_layout(
    title="Treasury Yield Curve Evolution",
    xaxis_title="Maturity (Years)",
    yaxis_title="Yield (%)",
    height=500,
    hovermode="x unified",
)

fig.show()

## 3. Build Bond Ladder

Create a 10-year bond ladder with equal allocations.

In [ ]:
from finbot.services.simulation.bond_ladder.bond import Bond
from finbot.services.simulation.bond_ladder.bond_ladder_simulator import BondLadderSimulator
from finbot.services.simulation.bond_ladder.ladder import Ladder

# Ladder parameters
total_investment = 100_000
ladder_length = 10  # 10 years
allocation_per_rung = total_investment / ladder_length

# Build ladder (equal allocation to each maturity)
bonds = []
for year in range(1, ladder_length + 1):
    bond = Bond(face_value=allocation_per_rung, maturity_years=year, purchase_date=pd.Timestamp("2020-01-01"))
    bonds.append(bond)

ladder = Ladder(bonds=bonds)

print("Bond Ladder Created:")
print(f"  Total Investment: ${total_investment:,.0f}")
print(f"  Number of Rungs: {ladder_length}")
print(f"  Allocation per Rung: ${allocation_per_rung:,.0f}")
print("\nLadder Structure:")
for i, bond in enumerate(bonds, 1):
    print(f"  Rung {i}: ${bond.face_value:,.0f} maturing in {bond.maturity_years} years")

## 4. Simulate Bond Ladder Performance

Run simulation to calculate present value and total return.

In [ ]:
# Initialize simulator
simulator = BondLadderSimulator(ladder=ladder, yield_curve_data=yield_df)

# Run simulation
print("Running bond ladder simulation...")
simulation_results = simulator.simulate(start_date="2020-01-01", end_date="2023-12-31")

print("\nSimulation Results:")
print(simulation_results.tail())

# Calculate key metrics
initial_value = simulation_results["total_value"].iloc[0]
final_value = simulation_results["total_value"].iloc[-1]
total_return = ((final_value / initial_value) - 1) * 100
years = len(simulation_results) / 252
annualized_return = ((final_value / initial_value) ** (1 / years) - 1) * 100

print("\nPerformance Summary:")
print("=" * 60)
print(f"  Initial Value: ${initial_value:,.0f}")
print(f"  Final Value: ${final_value:,.0f}")
print(f"  Total Return: {total_return:.2f}%")
print(f"  Annualized Return: {annualized_return:.2f}%")
print(f"  Period: {years:.2f} years")

## 5. Compare with Bond ETFs

Compare bond ladder performance against popular bond ETFs.

In [ ]:
from finbot.utils.data_collection_utils.yfinance.get_history import get_history

# Load bond ETFs for comparison
bond_etfs = {"TLT": "20+ Year Treasury", "IEF": "7-10 Year Treasury", "SHY": "1-3 Year Treasury"}

etf_data = {}

for symbol, name in bond_etfs.items():
    try:
        data = get_history(symbol, adjust_price=True)
        etf_data[symbol] = data
        print(f"✓ Loaded {symbol} ({name}): {len(data)} days")
    except Exception as e:
        print(f"✗ Error loading {symbol}: {e}")

# Align dates with simulation period
sim_dates = simulation_results.index
comparison_data = pd.DataFrame(index=sim_dates)
comparison_data["Ladder"] = simulation_results["total_value"]

for symbol in etf_data:
    # Normalize to same starting value
    etf_prices = etf_data[symbol]["Close"]
    aligned = etf_prices.reindex(sim_dates, method="ffill")
    normalized = aligned / aligned.iloc[0] * initial_value
    comparison_data[symbol] = normalized

# Calculate returns for each
print("\nPerformance Comparison:")
print("=" * 80)
for col in comparison_data.columns:
    start_val = comparison_data[col].iloc[0]
    end_val = comparison_data[col].iloc[-1]
    ret = ((end_val / start_val) - 1) * 100
    ann_ret = ((end_val / start_val) ** (1 / years) - 1) * 100
    print(f"  {col:10s}: Total Return = {ret:>6.2f}% | Annualized = {ann_ret:>6.2f}%")

## 6. Visualize Performance Comparison

Plot bond ladder vs ETF alternatives over time.

In [ ]:
# Create comparison chart
fig = go.Figure()

colors = {"Ladder": "blue", "TLT": "red", "IEF": "green", "SHY": "orange"}

for col in comparison_data.columns:
    fig.add_trace(
        go.Scatter(
            x=comparison_data.index,
            y=comparison_data[col],
            mode="lines",
            name=col,
            line=dict(color=colors.get(col, "gray"), width=2),
        )
    )

fig.update_layout(
    title="Bond Ladder vs Bond ETF Performance",
    xaxis_title="Date",
    yaxis_title="Portfolio Value ($)",
    height=600,
    hovermode="x unified",
)

fig.show()

## 7. Yield Curve Shape Analysis

Analyze how different yield curve shapes affect ladder performance.

In [ ]:
# Classify yield curve shapes
def classify_curve_shape(row):
    """Classify yield curve as normal, flat, or inverted."""
    if pd.isna(row["1-Year"]) or pd.isna(row["10-Year"]):
        return "Unknown"

    spread = row["10-Year"] - row["1-Year"]

    if spread > 1.0:
        return "Steep"
    elif spread > 0.25:
        return "Normal"
    elif spread > -0.25:
        return "Flat"
    else:
        return "Inverted"


yield_df["curve_shape"] = yield_df.apply(classify_curve_shape, axis=1)

# Count occurrences
shape_counts = yield_df["curve_shape"].value_counts()

print("Yield Curve Shape Distribution:")
print("=" * 60)
for shape, count in shape_counts.items():
    pct = (count / len(yield_df)) * 100
    print(f"  {shape:10s}: {count:>6,} days ({pct:>5.1f}%)")

# Plot 10Y-1Y spread over time
fig = go.Figure()

spread = yield_df["10-Year"] - yield_df["1-Year"]

fig.add_trace(
    go.Scatter(x=spread.index, y=spread.values, mode="lines", name="10Y-1Y Spread", line=dict(color="blue", width=1))
)

# Add reference lines
fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Inversion Threshold")

fig.update_layout(
    title="Treasury Yield Curve Slope (10Y - 1Y)", xaxis_title="Date", yaxis_title="Spread (%)", height=500
)

fig.show()

## Key Findings

From bond ladder analysis:

1. **Interest Rate Risk Management**: Bond ladders provide natural diversification across the yield curve, reducing sensitivity to any single maturity point

2. **Reinvestment Opportunity**: Regular maturities allow reinvestment at current rates, which can be advantageous in rising rate environments

3. **Liquidity Premium**: Bond ladders provide predictable liquidity at maturity dates without need to sell in secondary market

4. **Performance vs ETFs**: 
   - Long duration ETFs (TLT) are more volatile but can outperform in falling rate environments
   - Short duration ETFs (SHY) are more stable but lower yielding
   - Intermediate ETFs (IEF) often match ladder performance with more liquidity

5. **Yield Curve Shapes**:
   - **Normal/Steep curves**: Ladders capture yield premium at longer maturities
   - **Flat curves**: Less advantage to extending duration
   - **Inverted curves**: Short duration may be preferred

6. **Cost Considerations**: 
   - Bond ladders: Transaction costs at purchase, held to maturity
   - Bond ETFs: Annual expense ratios (typically 0.03-0.15%), potential bid-ask spreads

7. **Tax Efficiency**: Individual bonds allow precise control over capital gains timing vs ETF distributions

## Next Steps

- Model different ladder construction strategies (barbell, bullet)
- Test reinvestment rules (what to do with maturing proceeds)
- Compare Treasury ladders vs corporate bond ladders
- Analyze tax-equivalent yields for municipal bond ladders
- Test optimal ladder length for different goals
- Model callable bonds and prepayment risk